In [4]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
# 新宿区の日本人の人口を求める
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#> 
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX cd-code:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/> 
PREFIX cd-attribute:<http://data.e-stat.go.jp/lod/ontology/crossDomain/attribute/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX sac:<http://data.e-stat.go.jp/lod/sac/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select  ?o 
where {
      ?s pc-measure-2010:population ?o ;
         cd-dimension:standardAreaCode / ic:市区町村 "新宿区"@ja ;
         cd-dimension-2016:sex cd-code-2016:sex-total ; 
         cd-dimension-2016:nationality cd-code-2016:nationality-japanese ; 
         pc-dimension-2010:area pc-code-2010:area-all ; 
         cd-dimension-2016:age cd-code-2016:age-total . 
}


Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '286398'}}]}}

In [5]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
# 新宿区の日本人の人口を求める
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#> 
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX cd-code:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/> 
PREFIX cd-attribute:<http://data.e-stat.go.jp/lod/ontology/crossDomain/attribute/> 
PREFIX cd-dimension-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/2016/> 
PREFIX cd-code-2016:<http://data.e-stat.go.jp/lod/ontology/crossDomain/code/2016/> 
PREFIX pc-measure-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/measure/2010/> 
PREFIX pc-dimension-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/dimension/2010/> 
PREFIX pc-code-2010:<http://data.e-stat.go.jp/lod/ontology/populationCensus/code/2010/> 
PREFIX sac:<http://data.e-stat.go.jp/lod/sac/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select  ?o
where {
      ?s pc-measure-2010:population ?o ;
         cd-dimension:standardAreaCode ?areacode.
      ?areacode  ic:市区町村  "新宿区"@ja.
      ?s cd-dimension-2016:nationality cd-code-2016:nationality-japanese ; 
         cd-dimension-2016:sex cd-code-2016:sex-total ; 
         pc-dimension-2010:area pc-code-2010:area-all ; 
         cd-dimension-2016:age cd-code-2016:age-total . 
}


{'head': {'vars': ['o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '286398'}}]}}

In [4]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?hoikusho_num
where{
    ?s ssds-measure-2016:J2503 ?hoikusho_num;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:市区町村  "新宿区"@ja.
}


Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['num']},
 'results': {'bindings': [{'num': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                   'type': 'typed-literal',
                                   'value': '36'}}]}}

In [6]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?taikijido
where{
    ?s ssds-measure-2016:J250502 ?taikijido;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:市区町村  "新宿区"@ja.
}


Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['taikijido']},
 'results': {'bindings': [{'taikijido': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                         'type': 'typed-literal',
                                         'value': '176'}}]}}

In [23]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select *
where{
        {
        ?s ssds-measure-2016:J2503 ?o;
        cd-dimension:standardAreaCode / ic:市区町村 "新宿区"@ja. 
            } union {
        ?s ssds-measure-2016:J2503 ?o;
        cd-dimension:standardAreaCode / ic:市区町村 "文京区"@ja. 
        }
}

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query

{'head': {'vars': ['s', 'o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '36'},
                           's': {'type': 'uri',
                                 'value': 'http://data.e-stat.go.jp/lod/dataset/systemOfSocialAndDemographicStatistics/d00200502/obs0000000000590677'}},
                          {'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#decimal',
                                 'type': 'typed-literal',
                                 'value': '28'},
                           's': {'type': 'uri',
                                 'value': 'http://data.e-stat.go.jp/lod/dataset/systemOfSocialAndDemographicStatistics/d00200502/obs0000000000590678'}}]}}

In [36]:
%endpoint http://data.e-stat.go.jp/lod/sparql/query
%display table
PREFIX ssds-measure-2016:<http://data.e-stat.go.jp/lod/ontology/systemOfSocialAndDemographicStatistics/measure/2016/>
PREFIX cd-dimension:<http://data.e-stat.go.jp/lod/ontology/crossDomain/dimension/> 
PREFIX ic:<http://imi.ipa.go.jp/ns/core/rdf#> 
select ?taikijido ?shiku
where{
    ?s ssds-measure-2016:J250502 ?taikijido;
        cd-dimension:standardAreaCode ?areacode.
     ?areacode ic:都道府県  "東京都"@ja;
         ic:市区町村 ?shiku.
}
limit 10

Endpoint set to: http://data.e-stat.go.jp/lod/sparql/query
Display: table

taikijido,shiku
181,武蔵野市
181,墨田区
181,府中市
47,狛江市
184,西東京市
19,青梅市
96,文京区
35,あきる野市
257,町田市
270,豊島区
